In [1]:
import os
for folder in ['anforande-201617.json', 'anforande-201516.json','anforande-201112.json', 'anforande-201213.json']:
    path = os.path.join(os.getcwd(), folder)
    
    print(len(os.listdir(path)))


12725
13777
11578
12139


In [2]:
import os
import json
import pandas as pd
import gzip
import codecs

def compress_and_export(folders):
    # Initialize an empty list to store the dataframes
    df_list = []
    for folder in folders:
        # Iterate over the files in the folder
        folder = os.path.join(os.getcwd(), folder)

        index = 0
        for file in os.listdir(folder):
            
            # Check if the file is a JSON file
            _, ext = os.path.splitext(file)
            if ext == '.json':
                # Read the data from the JSON file
                with codecs.open(os.path.join(folder, file), 'r', encoding='utf-8-sig') as f:
                    data = json.load(f)

                data = data['anforande']
                
                if data['anforandetext'] is None:
                    continue

                data = {
                    'document_id': data['dok_id'] + '-' + data['anforande_nummer'],
                    'text': data['anforandetext'],
                    'party': data['parti'],
                    'year': data['dok_rm'],
                    'date': data['dok_datum'].split()[0]
                }

                # Convert the data to a dataframe
                tmp_idx = [index]
                df = pd.DataFrame(data, index=tmp_idx)
                index += 1

                # Append the dataframe to the list
                df_list.append(df)

        # Concatenate all of the dataframes into a single dataframe
    df = pd.concat(df_list, ignore_index=True)

    with gzip.open('speeches-full.json.gz', 'wt') as f:
        # Write the dataframe to the file in JSON format
        f.write(df.to_json(orient='records', lines=True))
            


In [3]:
compress_and_export([
    'anforande-201112.json',
    'anforande-201213.json',
    'anforande-201314.json',
    'anforande-201415.json',
    'anforande-201516.json',
    'anforande-201617.json'
])